In [6]:
import os
import json
from utils import load_datasets, load_target
import models
from models.tuning import beyesian_optimization
from models.evaluation import cross_validation_score
import json

In [7]:
config = json.load(open('./config/default.json'))
# X_train, X_test = load_datasets(["Age", "AgeSplit", "EducationNum"])
X_train, X_test = load_datasets(config['features'])
y_train = load_target('Y')


In [8]:
lgbm1 = models.Lgbm(json.load(open('./config/Lgbm-depth-5.json')))
lgbm2 = models.Lgbm(json.load(open('./config/Lgbm-depth-15.json')))
lgbm3 = models.Lgbm(json.load(open('./config/Lgbm-depth-inf.json')))
nn1 = models.NN(json.load(open('./config/NN-shallow.json')))
nn2 = models.NN(json.load(open('./config/NN-deep.json')))
rf = models.RandomForest(json.load(open('./config/RandomForest.json')))
ert = models.RandomForest(json.load(open('./config/ERT.json')))


In [9]:
model_list = [lgbm1, lgbm2, lgbm3, nn1, nn2, rf, ert]

In [10]:
from models.evaluation import stacking

oof, y_preds = stacking(model_list, X_train, y_train, X_test, 1)

/Users/yutahirai/opt/anaconda3/envs/tensorflow26/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/yutahirai/opt/anaconda3/envs/tensorflow26/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yutahirai/opt/anaconda3/envs/tensorflow26/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/yutahirai/opt/anaconda3/envs/tensorflow26/lib/python3.8/site-packages/lightgbm/engine.py:239: 

In [11]:
y_pred = [(i > 0.5).astype(int) for i in y_preds]
y_pred = sum(y_pred) / len(y_pred)
y_pred = (y_pred > 0.5).astype(int)
import pandas as pd
sample_submit = pd.read_csv(
    './data/input/sample_submit.csv', names=['id', 'Y'])
sample_submit['Y'] = y_pred
sample_submit.to_csv('./data/output/2022-2-19-submit.csv', header=False, index=False)
